## Logging on

Use Selenium to visit https://webapps1.chicago.gov/buildingrecords/ and accept the agreement.

> Think about when you use `.find_element...` and when you use `.find_elementSSS...`

In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from webdriver_manager.chrome import ChromeDriverManager

/Users/ilenapeng/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/ilenapeng/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/9x/ttxn_gwj0592mr6h8s5y29980000gn/T/ipykernel_88197/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
driver.get("https://webapps1.chicago.gov/buildingrecords/")

In [4]:
# select agreement button
driver.find_element(By.ID, 'rbnAgreement1').click()

In [5]:
# click submit
driver.find_element(By.ID, 'submit').click()

## Searching

Search for **400 E 41ST ST**.

In [6]:
# search address
driver.find_element(By.ID, 'fullAddress').send_keys('400 E 41ST ST')

In [7]:
# click submit
driver.find_element(By.ID, 'submit').click()

## Saving tables with pandas

Use pandas to save a CSV of all **permits** to `Permits - 400 E 41ST ST.csv`. Note that there are **different sections of the page**, not just one long permits table.

> - *Tip: When using `.read_html`, try using `flavor='lxml'` and comparing the results to `flavor='html5lib'`. Which works better?*
> - *Tip: You might need to install `html5lib` using `pip`. If so, you'll need to restart the notebook using **Kernel > Restart** before it will work.*

In [8]:
# grab table off of the page
table = driver.find_element(By.ID, 'resultstable_permits')

# then feed html using .get
df = pd.read_html(table.get_attribute('outerHTML'))[0]
df.head()

,PERMIT #,DATE ISSUED,DESCRIPTION OF WORK
0,100845718,NaN,ERECT TWO SCAFFOLDS FROM 10/14/2019 TO 10/14/2...
1,100778302,NaN,PERMIT EXPIRES ON 10/17/2018 Erection Starts: ...
2,100721255,NaN,PERMIT EXPIRES ON 10/24/2017 ERECTION STARTS: ...
3,100693399,NaN,INSTALLATION OF LOW VOLTAGE BURGLAR ALARM INTE...
4,100665436,NaN,PERMIT EXPIRES ON 10/24/2016 ERECTION STARTS: ...


In [9]:
# try with lxml
df = pd.read_html(table.get_attribute('outerHTML'), flavor='lxml')[0]
df.head()

,PERMIT #,DATE ISSUED,DESCRIPTION OF WORK
0,100845718,NaN,ERECT TWO SCAFFOLDS FROM 10/14/2019 TO 10/14/2...
1,100778302,NaN,PERMIT EXPIRES ON 10/17/2018 Erection Starts: ...
2,100721255,NaN,PERMIT EXPIRES ON 10/24/2017 ERECTION STARTS: ...
3,100693399,NaN,INSTALLATION OF LOW VOLTAGE BURGLAR ALARM INTE...
4,100665436,NaN,PERMIT EXPIRES ON 10/24/2016 ERECTION STARTS: ...


In [10]:
# try with html5lib -- this works best because the dates aren't NaNs!
df = pd.read_html(table.get_attribute('outerHTML'), flavor='html5lib')[0]
df.head()

,PERMIT #,DATE ISSUED,DESCRIPTION OF WORK
0,100845718,10/11/2019,ERECT TWO SCAFFOLDS FROM 10/14/2019 TO 10/14/2...
1,100778302,08/17/2018,PERMIT EXPIRES ON 10/17/2018 Erection Starts: ...
2,100721255,08/24/2017,PERMIT EXPIRES ON 10/24/2017 ERECTION STARTS: ...
3,100693399,03/03/2017,INSTALLATION OF LOW VOLTAGE BURGLAR ALARM INTE...
4,100665436,08/24/2016,PERMIT EXPIRES ON 10/24/2016 ERECTION STARTS: ...


In [11]:
df.to_csv('Permits - 400 E 41ST ST.csv', index=False)

## Saving tables the long way

Save a CSV of all DOB inspections to `Inspections - 400 E 41ST ST.csv`.

This is more complicated than the last one becuse **we also need to save the URL to the inspection** (see how the inspection number is a link?). As a result, you won't be able to use pandas! Instead, you'll need to use a loop and create a list of dictionaries.

You can use Selenium or you can feed the source to BeautifulSoup. You should have approximately 160 rows.

You'll probably need to find the table first, then the rows inside, then the cells inside of each row. You'll probably use lots of list indexing. I might recommend XPath for finding the table.

*Tip: If you get a "list index out of range" error, it's probably due to an issue involving `thead` vs `tbody` elements. What are they? What are they for? What's in them? There are a few ways to troubleshoot it.*

In [12]:
from bs4 import BeautifulSoup
import requests

In [13]:
doc = BeautifulSoup(driver.page_source)

In [14]:
# Finding everything that will show up in the dictionary

# full table row
# doc.find_all('table', id='resultstable_inspections')[0].find_all('tr')[1]

# INSP#
# doc.find_all('table', id='resultstable_inspections')[0].find_all('tr')[1].a.text

# Link
# doc.find_all('table', id='resultstable_inspections')[0].find_all('tr')[1].a['href']

# inspection date
# doc.find_all('table', id='resultstable_inspections')[0].find_all('tr')[1].span.text

# status
# doc.find_all('table', id='resultstable_inspections')[0].find_all('tr')[1].find_all('td')[-2].text

# type description
# doc.find_all('table', id='resultstable_inspections')[0].find_all('tr')[1].find_all('td')[-1].text

In [15]:
# the working for loop

# for row in doc.find_all('table', id='resultstable_inspections')[0].find_all('tr')[1:]:
#     print(row.a.text)
#     print(row.a['href'])
#     print(row.span.text)
#     print(row.find_all('td')[-2].text)
#     print(row.find_all('td')[-1].text)

In [16]:
dataset = []

for row in doc.find_all('table', id='resultstable_inspections')[0].find_all('tr')[1:]:
    data = {}
    data['insp_num'] = row.a.text
    data['link'] = row.a['href']
    data['insp_date'] = row.span.text
    data['status'] = row.find_all('td')[-2].text
    data['type'] = row.find_all('td')[-1].text
    dataset.append(data)

dataset

[{'insp_num': '13587202',
  'link': '/buildingrecords/inspectiondetails?addr=364923&insp=13587202',
  'insp_date': '20211025',
  'status': 'FAILED',
  'type': 'ANNUAL INSPECTION'},
 {'insp_num': '12941650',
  'link': '/buildingrecords/inspectiondetails?addr=364923&insp=12941650',
  'insp_date': '20210823',
  'status': 'FAILED',
  'type': 'CONSERVATION ANNUAL'},
 {'insp_num': '13528258',
  'link': '/buildingrecords/inspectiondetails?addr=364923&insp=13528258',
  'insp_date': '20210809',
  'status': 'FAILED',
  'type': 'ANNUAL INSPECTION'},
 {'insp_num': '13346279',
  'link': '/buildingrecords/inspectiondetails?addr=364923&insp=13346279',
  'insp_date': '20210303',
  'status': 'FAILED',
  'type': 'ANNUAL INSPECTION'},
 {'insp_num': '13175960',
  'link': '/buildingrecords/inspectiondetails?addr=364923&insp=13175960',
  'insp_date': '20201130',
  'status': 'FAILED',
  'type': 'ANNUAL INSPECTION'},
 {'insp_num': '12770690',
  'link': '/buildingrecords/inspectiondetails?addr=364923&insp=1277

In [17]:
df.to_csv('Inspections - 400 E 41ST ST.csv', index=False)

### Bonus preview of Wednesday's content

**You prrrrrobably shouldn't do this one unless you want a real challenge.**

If you click the inspection number, it'll open up a new window that shows you details of the violations from that visit. Count the number of violations for each visit and save it in a new column called **num_violations**.

Save this file as `Inspections - 400 E 41ST ST - with counts.csv`.

Since clicking the link opens in a new window, we have to say "Hey Selenium, pay attention to that new window!" We do that with `driver.switch_to.window(driver.window_handles[-1])` (each window gets a `window_handle`, and we're just asking the driver to switch to the last one.). A rough sketch of what your code will look like is here:

```python
# Click the link that opens the new window

# Switch to the new window/tab
driver.switch_to.window(driver.window_handles[-1])

# Do your scraping in here

# Close the new window/tab
driver.close()

# Switch back to the original window/tab
driver.switch_to.window(driver.window_handles[0])
```

You'll want to play around with them individually before you try it with the whole set - the ones that pass are very different pages than the ones with violations! There are a few ways to get the number of violations, some easier than others.